In [1]:
print("hello")

hello


In [2]:
import os
os.chdir('../')

In [3]:
from src.utils.minio_uploader import MinioUploader
from src.utils.delta_exporter import *
from src.components.data_ingestion import *
import logging
import csv
from datetime import datetime, timedelta, timezone
from binance_sdk_c2c.c2c import C2C, ConfigurationRestAPI, C2C_REST_API_PROD_URL
from turtle import home
from typing import List, Optional
from binance_sdk_c2c.rest_api.models import GetC2CTradeHistoryResponse, GetC2CTradeHistoryResponseDataInner
from src.utils.utils import *
from src.components.data_ingestion import C2CExtended

# Test

In [4]:
from binance_common.configuration import ConfigurationRestAPI
# Create configuration for the REST API
configuration_rest_api = ConfigurationRestAPI(
    api_key=os.getenv("API_KEY", ""),
    api_secret=os.getenv("API_SECRET", ""),
    base_path=os.getenv("BASE_PATH", C2C_REST_API_PROD_URL),
)
c2c = C2CExtended(configuration_rest_api)
# Ví dụ 5: Lấy giao dịch khoảng thời gian tùy chỉnh (01/09/2025 - 10/09/2025)
# Lấy giao dịch cho danh sách các ngày bất kỳ
data = c2c.get_latest_by_week()

INFO:root:Page 1 rate limits: []
INFO:root:Page 1 retrieved 50 records, 50 within time range
INFO:root:Page 2 rate limits: []
INFO:root:Page 2 retrieved 50 records, 0 within time range
INFO:root:Consecutive empty pages: 1/3
INFO:root:Page 3 rate limits: []
INFO:root:Page 3 retrieved 50 records, 0 within time range
INFO:root:Consecutive empty pages: 2/3
INFO:root:Page 4 rate limits: []
INFO:root:Page 4 retrieved 50 records, 0 within time range
INFO:root:Consecutive empty pages: 3/3
INFO:root:Stopping early: 3 consecutive pages with no data in time range


## Convert API's response data to deltalake

In [5]:
len(data)

50

In [6]:
# 1) Convert API results (list[dict] or objects) to DataFrame
df = api_results_to_dataframe(data)


2025-10-21 11:31:33.476 | INFO     | src.utils.delta_exporter:api_results_to_dataframe:50 - Created DataFrame: shape=(50, 18)


In [7]:
df.shape

(50, 18)

In [8]:
delta_path = "c2c"

In [9]:
write_dataframe_to_delta(df, delta_path, mode="append", partition_by=None)

2025-10-21 11:31:56.840 | INFO     | src.utils.delta_exporter:write_dataframe_to_delta:85 - Wrote DataFrame to Delta: path=c2c, mode=append, rows=50


'c2c'

In [10]:
test_bucket = "bronze"

In [11]:
uploader = MinioUploader(endpoint = "localhost:9000", access_key = "adminc2c", secret_key = "adminc2c", secure = False, region = None)

In [12]:
upload_delta_to_minio(delta_path = delta_path, uploader = uploader, bucket = test_bucket, dest_prefix = "c2c_trades")

2025-10-21 11:32:46.325 | INFO     | src.utils.minio_uploader:ensure_bucket:30 - Created bucket: bronze


2025-10-21 11:32:46.409 | INFO     | src.utils.minio_uploader:upload_file:57 - Uploaded file d:\Working\binance-merchant-trading-flow\c2c\part-00001-c40d3cd0-d35d-474f-af42-0a047b84331a-c000.snappy.parquet -> s3://bronze/c2c_trades/part-00001-c40d3cd0-d35d-474f-af42-0a047b84331a-c000.snappy.parquet
2025-10-21 11:32:46.471 | INFO     | src.utils.minio_uploader:upload_file:57 - Uploaded file d:\Working\binance-merchant-trading-flow\c2c\_delta_log\00000000000000000000.json -> s3://bronze/c2c_trades/_delta_log/00000000000000000000.json
2025-10-21 11:32:46.474 | INFO     | src.utils.delta_exporter:upload_delta_to_minio:109 - Uploaded Delta folder to s3://bronze/c2c_trades


# Spark Transformation - Đọc và xử lý Parquet files

In [4]:
from src.utils.spark_session import *


In [5]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("P2P_Binance_Bronze_to_Silver")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

print("✅ SparkSession initialized")

# Set log level to reduce noise
spark.sparkContext.setLogLevel("WARN")

print("SparkSession initialized successfully!")
print(f"Spark version: {spark.version}")

✅ SparkSession initialized
SparkSession initialized successfully!
Spark version: 3.5.3


In [6]:
# Đọc file Parquet từ Bronze zone
# Giả sử schema đã infer từ Parquet (nếu không, thêm .option("inferSchema", "true"))
df_bronze = spark.read.parquet("test_batch_transformation/delta/c2c_trades/data.parquet")

# Hiển thị schema và sample data để kiểm tra
df_bronze.printSchema()
df_bronze.show(5, truncate=False)

# Đếm tổng số records
print(f"Total records in Bronze: {df_bronze.count()}")

root
 |-- orderNumber: string (nullable = true)
 |-- advNo: string (nullable = true)
 |-- tradeType: string (nullable = true)
 |-- asset: string (nullable = true)
 |-- fiat: string (nullable = true)
 |-- fiatSymbol: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- totalPrice: string (nullable = true)
 |-- unitPrice: string (nullable = true)
 |-- orderStatus: string (nullable = true)
 |-- createTime: long (nullable = true)
 |-- commission: string (nullable = true)
 |-- counterPartNickName: string (nullable = true)
 |-- takerCommissionRate: string (nullable = true)
 |-- takerCommission: string (nullable = true)
 |-- takerAmount: string (nullable = true)
 |-- payMethodName: string (nullable = true)
 |-- additionalKycVerify: long (nullable = true)

+--------------------+--------------------+---------+-----+----+----------+-------------+-----------------+---------+-----------+-------------+----------+-------------------+-------------------+---------------+-----------+----

In [7]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

In [11]:
from decimal import Decimal  # Import cho lit(Decimal(...))

# Transformations cho Silver:
# 1. Cast types: createTime -> TIMESTAMP (từ ms Unix), amounts/prices -> DECIMAL
# 2. Handle nulls: advertisement_role -> 'UNKNOWN' (derive vì không có trong schema)
# 3. Enrich: trade_date (DATE từ createTime), total_vnd (= totalPrice), net_volume (BUY: +amount, SELL: -amount)
# 4. Standardize: tradeType uppercase, asset lowercase
# 5. Filter optional: Chỉ giữ COMPLETED (bỏ comment nếu cần)
# Note: Sử dụng camelCase từ schema; alias sang snake_case cho dễ dùng sau

df_silver = (df_bronze
    # Cast createTime từ long ms sang TIMESTAMP (không cần cast long nữa, đã là long)
    .withColumn("createTime", from_unixtime(col("createTime") / 1000))  # Convert ms to seconds, then to TIMESTAMP
    
    # Extract trade_date
    .withColumn("trade_date", to_date(col("createTime")))
    
    # Cast numeric columns (handle string to DECIMAL)
    .withColumn("amount", col("amount").cast(DecimalType(18, 8)))
    .withColumn("totalPrice", col("totalPrice").cast(DecimalType(18, 2)))
    .withColumn("unitPrice", col("unitPrice").cast(DecimalType(18, 2)))
    .withColumn("commission", coalesce(col("commission").cast(DecimalType(18, 8)), lit(Decimal("0.00000000"))))  # Dùng string cho Decimal literal
    
    # Enrich columns
    .withColumn("total_vnd", col("totalPrice"))  # Alias cho dễ query
    .withColumn("net_volume", 
                when(col("tradeType") == "BUY", col("amount"))
                .when(col("tradeType") == "SELL", -col("amount"))
                .otherwise(lit(Decimal("0.00000000")))  # Dùng string để tránh lỗi import
    )
    
    # Handle nulls và standardize
    .withColumn("tradeType", upper(col("tradeType")))
    .withColumn("asset", lower(col("asset")))
    
    # Optional: Filter chỉ COMPLETED trades
    # .filter(col("orderStatus") == "COMPLETED")
    
    # Select chỉ columns cần thiết (alias sang snake_case; derive advertisement_role)
    .select(
        col("orderNumber").alias("order_number"),
        col("advNo").alias("adv_no"),
        col("tradeType").alias("trade_type"),
        col("asset"),
        col("fiat"),
        col("fiatSymbol").alias("fiat_symbol"),
        col("amount"),
        col("totalPrice").alias("total_price"),
        col("unitPrice").alias("unit_price"),
        col("orderStatus").alias("order_status"),
        col("createTime").alias("create_time"),
        col("trade_date"),
        col("commission"),
        col("counterPartNickName").alias("counter_part_nick_name"),
        lit("UNKNOWN").alias("advertisement_role"),  # Derive vì không có trong schema
        col("total_vnd"),
        col("net_volume")
    )
)

# Kiểm tra schema mới và sample
df_silver.printSchema()
df_silver.show(5, truncate=False)

# Validate data quality: % nulls cho từng cột (sử dụng count để tránh lỗi nếu có nulls)
df_silver.select([round((count(when(col(c).isNull(), c)) / count("*")) * 100, 2).alias(c + "_null_pct") for c in df_silver.columns]).show()

root
 |-- order_number: string (nullable = true)
 |-- adv_no: string (nullable = true)
 |-- trade_type: string (nullable = true)
 |-- asset: string (nullable = true)
 |-- fiat: string (nullable = true)
 |-- fiat_symbol: string (nullable = true)
 |-- amount: decimal(18,8) (nullable = true)
 |-- total_price: decimal(18,2) (nullable = true)
 |-- unit_price: decimal(18,2) (nullable = true)
 |-- order_status: string (nullable = true)
 |-- create_time: string (nullable = true)
 |-- trade_date: date (nullable = true)
 |-- commission: decimal(18,8) (nullable = false)
 |-- counter_part_nick_name: string (nullable = true)
 |-- advertisement_role: string (nullable = false)
 |-- total_vnd: decimal(18,2) (nullable = true)
 |-- net_volume: decimal(18,8) (nullable = true)

+--------------------+--------------------+----------+-----+----+-----------+-------------+-----------+----------+------------+-------------------+----------+----------+----------------------+------------------+-----------+--------

# Direct Upload to Minio by Spark, Partitioning supported

In [6]:
# Cell 1: Imports và config logging
import logging
import os
from datetime import datetime

# Import PySpark và Delta
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, to_date, year, month, dayofmonth
from pyspark.sql.types import DecimalType

# Configure logging
logging.basicConfig(level=logging.INFO)

In [7]:
# Cell 2: Tạo SparkSession với Delta và MinIO config
# Thay giá trị MinIO thực của bạn
minio_endpoint = "localhost:9000"  # Thay bằng endpoint thực
minio_access_key = "adminc2c"  # Thay bằng access key thực
minio_secret_key = "adminc2c"  # Thay bằng secret key thực
minio_secure = False

spark = SparkSession.builder \
    .appName("Direct Delta Write to MinIO Bronze") \
    .config("spark.hadoop.fs.s3a.endpoint", f"http://{minio_endpoint}") \
    .config("spark.hadoop.fs.s3a.access.key", minio_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", minio_secret_key) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false" if not minio_secure else "true") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")
logging.info("SparkSession created with MinIO and Delta config.")

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1139)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1125)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:489)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1790)
	at org.apache.spark.SparkContext.$anonfun$new$16(SparkContext.scala:528)
	at org.apache.spark.SparkContext.$anonfun$new$16$adapted(SparkContext.scala:528)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:528)
	at org.apache.spark.api.java.JavaSparkC

In [12]:
from binance_common.configuration import ConfigurationRestAPI
# Create configuration for the REST API
configuration_rest_api = ConfigurationRestAPI(
    api_key=os.getenv("API_KEY", ""),
    api_secret=os.getenv("API_SECRET", ""),
    base_path=os.getenv("BASE_PATH", C2C_REST_API_PROD_URL),
)
c2c = C2CExtended(configuration_rest_api)
# Ví dụ 5: Lấy giao dịch khoảng thời gian tùy chỉnh (01/09/2025 - 10/09/2025)
# Lấy giao dịch cho danh sách các ngày bất kỳ
data = c2c.get_yesterday()

INFO:root:Page 1 rate limits: []
INFO:root:Page 1 retrieved 50 records, 50 within time range
INFO:root:Page 2 rate limits: []
INFO:root:Page 2 retrieved 50 records, 0 within time range
INFO:root:Consecutive empty pages: 1/3
INFO:root:Page 3 rate limits: []
INFO:root:Page 3 retrieved 50 records, 0 within time range
INFO:root:Consecutive empty pages: 2/3
INFO:root:Page 4 rate limits: []
INFO:root:Page 4 retrieved 50 records, 0 within time range
INFO:root:Consecutive empty pages: 3/3
INFO:root:Stopping early: 3 consecutive pages with no data in time range


In [13]:
len(data)

50

In [14]:
if not data:
    logging.warning("No data retrieved. Skipping.")
else:
    # Chuyển thành Pandas DF rồi sang Spark DF (vì api_results_to_dataframe dùng Pandas)
    pandas_df = api_results_to_dataframe(data)
    df_bronze = spark.createDataFrame(pandas_df)
    df_bronze.show(5, truncate=False)
    logging.info(f"DataFrame created with {df_bronze.count()} rows.")

2025-10-21 22:04:12.847 | INFO     | src.utils.delta_exporter:api_results_to_dataframe:50 - Created DataFrame: shape=(50, 18)


Py4JJavaError: An error occurred while calling o88.showString.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'spark_catalog': org.apache.spark.sql.delta.catalog.DeltaCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1926)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.loadV2SessionCatalog(CatalogManager.scala:68)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$2(CatalogManager.scala:87)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$1(CatalogManager.scala:87)
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.sql.connector.catalog.CatalogManager.v2SessionCatalog(CatalogManager.scala:86)
	at org.apache.spark.sql.connector.catal